In [1]:
//init cell

//common functions

const os = require('os');
const fs = require('fs');
const osc = require('osc');
const net = require('net');

function sliceStartStop(loop) {
    return loop.slice(1, -1)
}

function normalizeTime(loop) {
    const start_time = loop[0].ts;
    const dur = loop.slice(-1)[0].ts - start_time;
    touch_pts = loop.map(({ts, pos}) => ({pos, ts: (ts-start_time)/dur}));
    return touch_pts;
}

function deduplicate(loop) { //for fixing touchOSC bug of "lagging Y" redundant pre-message
    let filteredLoop = loop.filter((elem, i, arr) => i%2 != 0); //throw out even indices
    return filteredLoop;
    //todo - this only works when messages are guaranteed ordered (local UDP, tcp)
    //for non-ordered messages, probably most robust is to just toss first couple values to
    //get rid of "cut start" from redundant message (or to move cursor near intened start before recording)
}

function flipY(loop) {
    return loop.map(({ts, pos: {x, y}}) => ({ts, pos: {x, y: 1-y}}))
}

function perfectLoop(cleanedLoop) {
    let startPt = cleanedLoop[0];
    let endPt = cleanedLoop.slice(-1)[0];
    let netDelta = {x: endPt.pos.x - startPt.pos.x, y: endPt.pos.y - startPt.pos.y};
    let numDelt = cleanedLoop.length - 1
    let delta = {x: netDelta.x/numDelt, y: netDelta.y/numDelt};
    let closedLoop = cleanedLoop.map(({ts, pos: {x, y}}, i)=> ({ts, pos: {x: x-i*delta.x, y: y-i*delta.y}}));
    return closedLoop;
}

function oscArgs(...args) {
    let output  = args.map(e => {
        let type_str = typeof e;
        switch(type_str) {
            case 'string':
                return {type: 's', value: e}
            case 'number': 
                return {type: 'f', value: e}
            case 'boolean':
                return {type: 'i', value: e+0}
        }
    });
    return output;
}

function saveLoops(fileName) {
    if(!fileName) fileName = Date().toString().replaceAll(/[\: \)\(]/g, "-") + ".json";
    fs.writeFileSync(fileName, JSON.stringify({rawLoops, processedLoops}));
}
function loadLoopFile(fileName) {
    return JSON.parse(fs.readFileSync(fileName));
}
function updateLoops() {
    sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ));
}
function loadLoops(fileName) {
    let loops = loadLoopFile(fileName);
    rawLoops = loops.rawLoops;
    processedLoops = loops.processedLoops;
    sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ));
    return new Promise((resolve, reject) => setTimeout(resolve, 150));
}


//TCP server to OF

const tcp_port = 11999;
const tcp_host = '127.0.0.1';

const server = net.createServer();
server.listen(tcp_port, tcp_host, () => {
    console.log('TCP Server is running on port ' + tcp_port + '.');
});

const sockets = [];

server.on('connection', function(sock) {
    console.log('CONNECTED: ' + sock.remoteAddress + ':' + sock.remotePort);
    sockets.push(sock);

    sock.write( JSON.stringify(processedLoops) + '\n' );

    sock.on('data', function(data) {
        console.log('DATA ' + sock.remoteAddress + ': ' + data);
        // Write the data back to all the connected, the client will receive it as data from the server
        sockets.forEach(function(sock, index, array) {
            sock.write(sock.remoteAddress + ':' + sock.remotePort + " said " + data + '\n');
        });
    });

    // Add a 'close' event handler to this instance of socket
    sock.on('close', function(data) {
        let index = sockets.findIndex(function(o) {
            return o.remoteAddress === sock.remoteAddress && o.remotePort === sock.remotePort;
        })
        if (index !== -1) sockets.splice(index, 1);
        console.log('CLOSED: ' + sock.remoteAddress + ' ' + sock.remotePort);
    });
});

//OSC Server to touchOSC

const TCP_OSC_PORT = 7071;
const TCP_OSC_HOST = '192.168.1.39';

const tcp_osc_port = new osc.TCPSocketPort({});
// change to remote host/port (address of phone/tablet) - set touchOSC on device to TCPserver
tcp_osc_port.open(TCP_OSC_HOST, TCP_OSC_PORT); 

tcp_osc_port.on('ready', () => {
  console.log('ready OSC TCP');
});


const UDP_OSC_PORT = 7071;
const UDP_OSC_HOST = '0.0.0.0';

// Bind to a UDP socket to listen for incoming OSC events.
const udp_osc_port = new osc.UDPPort({
    localAddress: UDP_OSC_HOST,
    localPort: UDP_OSC_PORT,
    remoteAddress: '127.0.0.1',
    remotePort: 7072
});

udp_osc_port.on("ready", () => {
    console.log('ready OSC UDP');
});

udp_osc_port.open();

function clearAll() {
    udp_osc_port.send({address: "/clearAll", args: []});
}

let osc_port = tcp_osc_port;

let rawLoops = {};
let processedLoops = {};
let activeLoop = 'firstLoop';
let isRecording = false;
let isTouching = false;
const handlers = {};

function processLoop(loopKey) {
    processedLoops[loopKey] = flipY(normalizeTime(deduplicate(sliceStartStop(rawLoops[loopKey]))));
}

handlers['/recording'] = (isRec) => {
    console.log("isRec", isRec);
    isRecording = isRec;
    if(isRecording) {
        rawLoops[activeLoop] = [{ts: Date.now(), pos: 'start'}];
    } else {
        rawLoops[activeLoop].push({ts: Date.now(), pos: 'end'});
        processLoop(activeLoop);
        sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ))
    }
}
const f = float => ({type: 'f', value: float}); //helper for formatting floats for OSC
handlers['/pos'] = (x, y) => {
    if(isRecording) {
        rawLoops[activeLoop].push({ts: Date.now(), pos: {x, y}});
    }
    udp_osc_port.send({address: '/touchPos', args: [f(x), f(y)]});
}
handlers['/pos/touch'] = (touching) => {
    isTouching = touching;
    udp_osc_port.send({address: '/touching', args: [f(isTouching)]});
}

let logAddresses = [];
osc_port.on('message', (message) => {
    let {address, args} = message;
    if(logAddresses.includes(address) || logAddresses.includes("all")) console.log(message);
    if(handlers[address]) handlers[address](...args); 
});

let loadFile = "";
if(loadFile) {
    console.log("loaded file", loadFile);
    let loops = loadLoops(loadFile);
    processedLoops = loops.processedLoops;
    rawLoops = loops.rawLoops;
}

"Setup Done"

'Setup Done'

ready OSC UDP
TCP Server is running on port 11999.
ready OSC TCP
CONNECTED: 127.0.0.1:64747


In [ ]:
//tight descending spirals
// udp_osc_port.send({address: "/gridSize", args: oscArgs(4)});
// for(var i = 0; i < 100; i++) {
//     launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop2', 10, false, 'grp1', 'key'+i)});
//     setTimeout(launch, i*50)
// }

//wide+short descending spirals
// udp_osc_port.send({address: "/gridSize", args: oscArgs(10)});
// for(var i = 0; i < 100; i++) {
//     launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop3', 30, false, 'grp1', 'key'+i)});
//     setTimeout(launch, i*150)
// }

5

In [23]:
function perfectLoop(cleanedLoop) {
    let startPt = cleanedLoop[0];
    let endPt = cleanedLoop.slice(-1)[0];
    let netDelta = {x: endPt.pos.x - startPt.pos.x, y: endPt.pos.y - startPt.pos.y};
    let numDelt = cleanedLoop.length - 1
    let delta = {x: netDelta.x/numDelt, y: netDelta.y/numDelt};
    let closedLoop = cleanedLoop.map(({ts, pos: {x, y}}, i)=> ({ts, pos: {x: x-i*delta.x, y: y-i*delta.y}}));
    return closedLoop;
}
function updateLoops() {
    sockets.forEach(sock => sock.write( JSON.stringify(processedLoops) + '\n' ));
}
updateLoops()

In [39]:
clearAll()
// udp_osc_port.send({address: "/gridSize", args: oscArgs(10)});
// loadLoops('test_loops.json')
// Object.keys(loop1)

// processedLoops["loop2_full"] = perfectLoop(processedLoops["loop2"])
// udp_osc_port.send({address: "/useVoronoi", args: oscArgs(1)});
// udp_osc_port.send({address: "/gridSize", args: oscArgs(4)});
// udp_osc_port.send({address: "/launch", args: oscArgs('loop2_full', 5, false, 'grp1', 'key2')});
// udp_osc_port.send({address: "/launch", args: oscArgs('loop2_full', 7, false, 'grp1', 'key2')});
// udp_osc_port.send({address: "/launch", args: oscArgs('loop2_full', 10, false, 'grp1', 'key2')});
// for(var i = 0; i < 10; i++) {
//     let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop2', 5, false, 'grp1', 'key2')});
//     setTimeout(launch, i*100)
// }
// for(var i = 0; i < 10; i++) {
//     let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop2_full', 10, false, 'grp1', 'key2')});
//     setTimeout(launch, i*100)
// }
// for(var i = 0; i < 10; i++) {
//     let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop2_full', 10, false, 'grp1', 'key2')});
//     setTimeout(launch, i*10)
// }

CLOSED: 127.0.0.1 50115


In [2]:
activeLoop = "loop2"
// udp_osc_port.send({address: "/gridSize", args: oscArgs(10)});
// udp_osc_port.send({address: "/useVoronoi", args: oscArgs(1)});

'loop2'

isRec 1
isRec 0


In [ ]:
//sketches
async function sketch1() {
    await loadLoops('test_loops.json')
    let dev = 500; //40,
    for(var i = 0; i < dev; i++) {
        let dur = 15 + i/dev
        udp_osc_port.send({address: "/launch", args: oscArgs('loop1', dur, false, 'grp1', 'key'+i)});
    }
}

async function sketch2() {
    await loadLoops('test_loops.json')
    let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 30, false, 'grp1', 'key1')});
    for(var i = 0; i < 100; i ++) {
        setTimeout(launch, i*10)
        setTimeout(launch, 4000 + i*10)
    }
}

async function sketch3() {
    udp_osc_port.send({address: "/useVoronoi", args: oscArgs(1)});
    await loadLoops('test_loops.json')
    let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 30, false, 'grp1', 'key1')});
    for(var i = 0; i < 10; i ++) {
        setTimeout(launch, i*100)
    }
}

async function sketch4() {
    udp_osc_port.send({address: "/gridSize", args: oscArgs(4)});
    udp_osc_port.send({address: "/useVoronoi", args: oscArgs(1)});
    await loadLoops('test_loops.json')
    let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 60, false, 'grp1', 'key1')});
    for(var i = 0; i < 10; i ++) {
        setTimeout(launch, i*100)
        setTimeout(launch, 4000 + i*100)
    }
}

In [ ]:
//basic ws template

const ws = require('ws')

const wss = new ws.Server({ port: 8080 });

wss.on('connection', function connection(ws) {
  console.log("connected")
  ws.on('message', function message(data) {
    console.log('received: %s', data);
  });
});